# A3a: Signals and Noise
Finn Hittson - fxh157  
CSDS 464  
Due: February 27th, 2023

## 1. Continuous signals and sampling

### 1a. Sampled functions

In [1]:
import A3a_fxh157 as a3a
import math
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('../464-A1b_fxh157_files/')
import A1b_fxh157 as a1b

a3a.plot_sampled_function(g=a1b.sinewave, fs=5, tlim=(0, 6), tscale=1, tunits="msec", f=1)

a3a.plot_sampled_function(g=a1b.gabore, fs=3, tlim=(-0.03, 0.03), tscale=0.01, tunits="ms", f=100, a=0.106, sigma=0.01)

a3a.plot_sampled_function(g=a1b.gammatone, fs=5, tlim=(0,0.04), tscale=0.01, tunits="ms", f=200)

### 1b. The Nyquist frequency and aliasing

a3a.plot_sampled_function(g=a1b.sinewave, fs=5, tlim=(0, 6), tscale=1.3, tunits="msec", f=1)

a3a.plot_sampled_function(g=a1b.sinewave, fs=2, tlim=(0, 6), tscale=1, tunits="msec", f=1)

a3a.plot_sampled_function(g=a1b.sinewave, fs=2, tlim=(0, 6), tscale=1, tunits="msec", f=1, d=0.25)

a3a.plot_sampled_function(g=a1b.sinewave, fs=6, tlim=(0, 10), tscale=1, tunits="msec", f=5, d=0.25/5)

## 2. Signals

### 2a. Delta and step functions

a3a.plot_delta_step(t=10, fs=2, g=a3a.d, plot_type="stem")

a3a.plot_delta_step(t=10, fs=2, g=a3a.u, plot_type="stem")

### 2b. `gensignal`

t, y = a3a.gensignal(t=0, g=a1b.gammatone, fs=100, tau=0, T=0.06, f=100)
a3a.plot_stem(t, y, title="Gammatone")

## 3. Noise and SNR

### 3a. `energy`, `power`, and `snr`

In [2]:
def energy(x):
    return np.linalg.norm(x)**2

def power(x):
    return np.linalg.norm(x)**2/len(x)

def snr(Ps, Pn):
    return Ps/Pn

### 3b. Noisy signals

t, y, n = a3a.noisysignal(t=0, g=a1b.sinewave, fs=1000, tau=0, T=5, s=0.2)
a3a.pl`ot_noisysignal(t, y+n, "Sinusodial")

t, y, n = a3a.noisysignal(t=-5, g=a3a.u, fs=500, tau=0, T=5, s=0.1)
a3a.plot_noisysignal(t, y+n, "Step Function")

t, y, n = a3a.noisysignal(t=0, g=a1b.gammatone, fs=1000, tau=0, T=0.06, s=0.000025, f=100)
a3a.plot_noisysignal(t, y+n, title="Gammatone")

### 3c. Noise level specified by SNR

In [3]:
sigma = 0
for _ in range(100):
    t, _, n = a3a.noisysignal(t=0, g=a1b.sinewave, fs=1000, tau=0, T=5, s=0.2)
    _, x, _ = a3a.noisysignal(t=0, g=a1b.sinewave, fs=1000, tau=0, T=5, s=0)
    Ps = a3a.power(x)
    Pn = a3a.power(n)
    SNR = a3a.snr(Ps, Pn)
    sigma += a3a.snr2sigma(x, snr=SNR)
print(sigma/100)

0.16671487391122886
